"google brain EDA Japanese and start"のコピーです。  
日本語訳の修正やEDAの追加をした物です。

**コンテストの概論**

患者が呼吸に問題がある場合、医師は人工呼吸器を使用して、気管内のチューブを介して鎮静状態の患者の肺に酸素を送り込みます。しかし、機械的人工呼吸は臨床医による集中治療であり、COVID-19パンデミックの初期に顕著に行われました。人工呼吸器の制御方法の開発は、臨床試験に到達する前にさえ、法外に費用がかかります。高品質のシミュレーターは、この障壁を減らすことができます。

現在のシミュレーターは、各モデルが一つの肺を模倣するアンサンブルとしてトレーニングされています。ただし、肺と肺に関連する臓器は連続した空間を形成するため、患者の肺の違いを考慮したパラメトリックアプローチを検討する必要があります。

プリンストン大学と提携して、Google Brainのチームは、機械的人工呼吸制御のための機械学習を中心としたコミュニティの成長を目指しています。彼らは、ニューラルネットワークとディープラーニングにより、PIDコントローラーの現在の業界標準よりも肺の一般化ができると信じています。

このコンペでは、鎮静化した患者の肺に接続された人工呼吸器をシミュレートします。最良の提出は、肺の属性の追随性と抵抗を考慮に入れます。

成功すれば、人工呼吸器を制御するための新しい方法を開発することのコスト障壁を克服するのに役立ちます。これにより、患者に適応し、これらの新しい時代以降の臨床医の負担を軽減するアルゴリズムへの道が開かれます。その結果、人工呼吸器治療は、患者の呼吸を助けるためにより広く利用できるようになる可能性があります。

このコンペの人工呼吸器データは、呼吸回路を介して人工ベローズテスト肺に接続されたオープンソース人工呼吸器を調整して作成されました。次の図は、構造図を示しています。  
2つの制御入力が緑色で強調表示され、状態変数（気道内圧）が青色で予測されています。最初の制御入力は、肺に空気を入れるために吸気電磁弁が開いているパーセンテージを表す0から100までの連続変数です（つまり、0は完全に閉じており、空気は入れず、100は完全に開いています）。2番目の制御入力は、空気を排出するために探索バルブが開いている（1）か閉じている（0）かを表すバイナリ変数です。

このコンペでは、参加者は多数の呼吸時系列データを与えられ、制御入力の時系列を前提として、呼吸中の呼吸回路の気道内圧を予測することを学びます。

人工呼吸器の図

![代替テキスト](https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg)




各時系列は、約3秒の呼吸を表します。ファイルは、各行が呼吸の時間ステップであり、2つの制御信号、結果として生じる気道内圧、および以下に説明する肺の関連属性を提供するように編成されています。

**人工呼吸器のデータを使って、気道内圧(pressure)を予測する。**

**Andrew Lukyanenko様のコード拝見（拝借）します。**

[https://www.kaggle.com/artgor/ventilator-pressure-prediction-eda-fe-and-models](https://www.kaggle.com/artgor/ventilator-pressure-prediction-eda-fe-and-models)

In [ ]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import numpy as np
import time
import lightgbm as lgb

from sklearn.model_selection import GroupKFold
from sklearn import metrics

**まずは、データを見ていきます。**

In [ ]:
path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))
sub = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [ ]:
train.head()

In [ ]:
train.describe()

**600万行ある**

In [ ]:
train

ファイル
- train.csv-トレーニングセット
- test.csv-テストセット
- sample_submission.csv-正しい形式のサンプル提出ファイル

列
- id：データのID  
    ファイル全体で一意のタイムステップ識別子
    
- breath_id：呼吸毎のID  
    呼吸のグローバルに一意のタイムステップ
    
- R：流量変化に対する圧力変化 (5,20,50)  
    気道の抵抗度合いを示す肺の属性（cmH2O / L / S）。  
    物理的には、これは流量の変化あたりの圧力の変化（時間あたりの空気量）です。  
    直感的には、ストローで風船を膨らませるイメージです。ストローの直径を変えることでRを変えることができますが、Rが大きいほど膨らませるのが大変になります。
    
- C：圧力変化に対する体積変化 (10,20,50)　  
    肺の追随度合いを示す肺の属性（mL / cmH2O）。  
    物理的には、これは圧力の変化ごとの体積の変化です。  
    直感的には、風船のゴムの厚さに似た概念がCだと思ってください。  
    Cが高いほど、ゴムの厚さは薄くなり、膨らみやすくなります。
    
- time_step：データ計測の時間  
    タイムスタンプ
- u_in：吸気弁の開き具合（0～100の連測値）   
    吸気電磁弁の制御入力。範囲は0〜100の連続値。

- u_out：呼気弁の開閉（0, 1の2値）  
    呼気電磁弁の制御入力。  open (1) or closed (0)のいずれか。

- pressure:気道内圧  
    呼吸回路で測定された気道内圧。cmH2Oで測定されます。

In [ ]:
print("testとtrainのbreath_idの重複:",set(test['breath_id'].unique()).intersection(set(train['breath_id'].unique())))

In [ ]:
fig, ax = plt.subplots(figsize = (12, 8))
plt.subplot(2, 2, 1)
sns.countplot(x='R', data=train)
plt.title('Counts of R in train');
plt.subplot(2, 2, 2)
sns.countplot(x='R', data=test)
plt.title('Counts of R in test');
plt.subplot(2, 2, 3)
sns.countplot(x='C', data=train)
plt.title('Counts of C in train');
plt.subplot(2, 2, 4)
sns.countplot(x='C', data=test)
plt.title('Counts of C in test');
plt.tight_layout()

**RとCは連続値と言いながらも、3つのカテゴリーに分かれていてほぼ均等に分けられている。**

In [ ]:
# 4呼吸分のデータを見てみる
fig = plt.figure(figsize = (16, 12))
for i in range(4):
    ax1=plt.subplot(2,2,i+1)
    breath_1 = train.loc[train['breath_id'] == i+1]
    ax2 = ax1.twinx()

    ax1.plot(breath_1['time_step'], breath_1['pressure'], 'r-', label='pressure')
    ax1.plot(breath_1['time_step'], breath_1['u_in'], 'g-', label='u_in')
    ax2.plot(breath_1['time_step'], breath_1['u_out'], 'b-', label='u_out')

    ax1.set_xlabel('Timestep')

    ax1.legend(bbox_to_anchor=(0.95, 0.95), loc="upper right")
    ax1.set_ylabel("pressure, u_in")
    ax2.legend(bbox_to_anchor=(0.95, 0.8), loc="upper right")
    ax2.set_ylabel("u_out")
    ax1.set_title(f"R:{breath_1['R'].unique()} C:{breath_1['C'].unique()}")
plt.show()

- u-inが大きくなった後、遅れてpressureが大きくなる 
- RとCによってu_inとpressureの関係性は変わってきそう
- u_outが1になっていてもu_inが0になっていないところがある

In [ ]:
# rewritten calculation of lag features from this notebook: https://www.kaggle.com/patrick0302/add-lag-u-in-as-new-feat
# some of ideas from this notebook: https://www.kaggle.com/mst8823/google-brain-lightgbm-baseline
train['last_value_u_in'] = train.groupby('breath_id')['u_in'].transform('last')
train['u_in_lag1'] = train.groupby('breath_id')['u_in'].shift(1)
train['u_out_lag1'] = train.groupby('breath_id')['u_out'].shift(1)
train['u_in_lag_back1'] = train.groupby('breath_id')['u_in'].shift(-1)
train['u_out_lag_back1'] = train.groupby('breath_id')['u_out'].shift(-1)
train['u_in_lag2'] = train.groupby('breath_id')['u_in'].shift(2)
train['u_out_lag2'] = train.groupby('breath_id')['u_out'].shift(2)
train['u_in_lag_back2'] = train.groupby('breath_id')['u_in'].shift(-2)
train['u_out_lag_back2'] = train.groupby('breath_id')['u_out'].shift(-2)
train['u_in_lag3'] = train.groupby('breath_id')['u_in'].shift(3)
train['u_out_lag3'] = train.groupby('breath_id')['u_out'].shift(3)
train['u_in_lag_back3'] = train.groupby('breath_id')['u_in'].shift(-3)
train['u_out_lag_back3'] = train.groupby('breath_id')['u_out'].shift(-3)
train = train.fillna(0)


train['R__C'] = train["R"].astype(str) + '__' + train["C"].astype(str)

# max value of u_in and u_out for each breath
train['breath_id__u_in__max'] = train.groupby(['breath_id'])['u_in'].transform('max')
train['breath_id__u_out__max'] = train.groupby(['breath_id'])['u_out'].transform('max')

# difference between consequitive values
train['u_in_diff1'] = train['u_in'] - train['u_in_lag1']
train['u_out_diff1'] = train['u_out'] - train['u_out_lag1']
train['u_in_diff2'] = train['u_in'] - train['u_in_lag2']
train['u_out_diff2'] = train['u_out'] - train['u_out_lag2']
# from here: https://www.kaggle.com/yasufuminakama/ventilator-pressure-lstm-starter
train.loc[train['time_step'] == 0, 'u_in_diff'] = 0
train.loc[train['time_step'] == 0, 'u_out_diff'] = 0

# difference between the current value of u_in and the max value within the breath
train['breath_id__u_in__diffmax'] = train.groupby(['breath_id'])['u_in'].transform('max') - train['u_in']
train['breath_id__u_in__diffmean'] = train.groupby(['breath_id'])['u_in'].transform('mean') - train['u_in']

# OHE
train = train.merge(pd.get_dummies(train['R'], prefix='R'), left_index=True, right_index=True).drop(['R'], axis=1)
train = train.merge(pd.get_dummies(train['C'], prefix='C'), left_index=True, right_index=True).drop(['C'], axis=1)
train = train.merge(pd.get_dummies(train['R__C'], prefix='R__C'), left_index=True, right_index=True).drop(['R__C'], axis=1)

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
train['u_in_cumsum'] = train.groupby(['breath_id'])['u_in'].cumsum()
train['time_step_cumsum'] = train.groupby(['breath_id'])['time_step'].cumsum()

**A氏は特徴量を増やしたみたい。**

In [ ]:
train

In [ ]:
# all the same for the test data
test['last_value_u_in'] = test.groupby('breath_id')['u_in'].transform('last')
test['u_in_lag1'] = test.groupby('breath_id')['u_in'].shift(1)
test['u_out_lag1'] = test.groupby('breath_id')['u_out'].shift(1)
test['u_in_lag_back1'] = test.groupby('breath_id')['u_in'].shift(-1)
test['u_out_lag_back1'] = test.groupby('breath_id')['u_out'].shift(-1)
test['u_in_lag2'] = test.groupby('breath_id')['u_in'].shift(2)
test['u_out_lag2'] = test.groupby('breath_id')['u_out'].shift(2)
test['u_in_lag_back2'] = test.groupby('breath_id')['u_in'].shift(-2)
test['u_out_lag_back2'] = test.groupby('breath_id')['u_out'].shift(-2)
test['u_in_lag3'] = test.groupby('breath_id')['u_in'].shift(3)
test['u_out_lag3'] = test.groupby('breath_id')['u_out'].shift(3)
test['u_in_lag_back3'] = test.groupby('breath_id')['u_in'].shift(-3)
test['u_out_lag_back3'] = test.groupby('breath_id')['u_out'].shift(-3)
test = test.fillna(0)
test['R__C'] = test["R"].astype(str) + '__' + test["C"].astype(str)

test['breath_id__u_in__max'] = test.groupby(['breath_id'])['u_in'].transform('max')
test['breath_id__u_out__max'] = test.groupby(['breath_id'])['u_out'].transform('max')

test['u_in_diff1'] = test['u_in'] - test['u_in_lag1']
test['u_out_diff1'] = test['u_out'] - test['u_out_lag1']
test['u_in_diff2'] = test['u_in'] - test['u_in_lag2']
test['u_out_diff2'] = test['u_out'] - test['u_out_lag2']
test.loc[test['time_step'] == 0, 'u_in_diff'] = 0
test.loc[test['time_step'] == 0, 'u_out_diff'] = 0

test['breath_id__u_in__diffmax'] = test.groupby(['breath_id'])['u_in'].transform('max') - test['u_in']
test['breath_id__u_in__diffmean'] = test.groupby(['breath_id'])['u_in'].transform('mean') - test['u_in']

test = test.merge(pd.get_dummies(test['R'], prefix='R'), left_index=True, right_index=True).drop(['R'], axis=1)
test = test.merge(pd.get_dummies(test['C'], prefix='C'), left_index=True, right_index=True).drop(['C'], axis=1)
test = test.merge(pd.get_dummies(test['R__C'], prefix='R__C'), left_index=True, right_index=True).drop(['R__C'], axis=1)

test['u_in_cumsum'] = test.groupby(['breath_id'])['u_in'].cumsum()
test['time_step_cumsum'] = test.groupby(['breath_id'])['time_step'].cumsum()

**テストも同じ**

# Model Training

**みんな大好きモデルをやってみよう**

In [ ]:
scores = []
feature_importance = pd.DataFrame()
models = []
columns = [col for col in train.columns if col not in ['id', 'breath_id', 'pressure']]
X = train[columns]
y = train['pressure']

In [ ]:
params = {'objective': 'regression',
          'learning_rate': 0.3,
          "boosting_type": "gbdt",
          "metric": 'mae',
          'n_jobs': -1,
          'min_data_in_leaf':32,
          'num_leaves':1024,
         }

In [ ]:
folds = GroupKFold(n_splits=5)
for fold_n, (train_index, valid_index) in enumerate(folds.split(train, y, groups=train['breath_id'])):
    print(f'Fold {fold_n} started at {time.ctime()}')
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    model = lgb.LGBMRegressor(**params, n_estimators=10000)
    model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            verbose=1000, early_stopping_rounds=15)
    score = metrics.mean_absolute_error(y_valid, model.predict(X_valid))
    
    models.append(model)
    scores.append(score)

    fold_importance = pd.DataFrame()
    fold_importance["feature"] = columns
    fold_importance["importance"] = model.feature_importances_
    fold_importance["fold"] = fold_n + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

In [ ]:
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

In [ ]:
feature_importance["importance"] /= 5
cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
    by="importance", ascending=False)[:50].index

best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

plt.figure(figsize=(16, 12));
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
plt.title('LGB Features (avg over folds)');

**答えまでできちゃいます<(_ _)>**

In [ ]:
for model in models:
    sub['pressure'] += model.predict(test[columns])
sub['pressure'] /= 5

In [ ]:
sub.to_csv('first_sub.csv', index=False)

In [ ]:
sub